In [119]:
user_name = "parasyte" # REPLACE THIS WITH YOUR HOST MACHINE USER NAME 
root_dir = f"/tmp/finn_dev_{user_name}"
print(root_dir)
# get onnx model from the last NOTEBOOK
filename = root_dir + "/ready_finn.onnx"

/tmp/finn_dev_parasyte


In [120]:
from finn.util.visualization import showSrc, showInNetron


In [125]:
# showInNetron(filename)

In [126]:
from qonnx.transformation.general import GiveReadableTensorNames, GiveUniqueNodeNames, RemoveStaticGraphInputs
from qonnx.transformation.infer_shapes import InferShapes
from qonnx.transformation.infer_datatypes import InferDataTypes
from qonnx.transformation.fold_constants import FoldConstants
from qonnx.core.modelwrapper import ModelWrapper

model = ModelWrapper(filename)

# TIDY UP
model = model.transform(InferShapes())
model = model.transform(FoldConstants())
model = model.transform(GiveUniqueNodeNames())
model = model.transform(GiveReadableTensorNames())
model = model.transform(InferDataTypes())
model = model.transform(RemoveStaticGraphInputs())

model.save(root_dir + "/tidy.onnx")

showSrc(InferShapes)

class InferShapes(Transformation):
    """Ensure every tensor in the model has a specified shape (ValueInfo)."""

    def apply(self, model):
        # hide your riches!
        hidden_ops = _hide_finn_ops(model)
        # call regular ONNX shape inference
        model = ModelWrapper(si.infer_shapes(model.model))
        # bring back hidden ops
        _restore_finn_ops(model, hidden_ops)
        return (model, False)



In [128]:
# showInNetron(root_dir + "/tidy.onnx")

In [129]:

from finn.util.pytorch import ToTensor
from qonnx.transformation.merge_onnx_models import MergeONNXModels
from qonnx.core.datatype import DataType
from brevitas.export import export_qonnx
from qonnx.util.cleanup import cleanup as qonnx_cleanup
import torch
from finn.transformation.qonnx.convert_qonnx_to_finn import ConvertQONNXtoFINN

# PRE PROC : NONE
model = ModelWrapper(root_dir + "/tidy.onnx")

#=========================================================
# Here you could add some proprocessing if you need some
#=========================================================

# add input annotation: UINT8 is what we will feed the model during inference
global_inp_name = model.graph.input[0].name
model.set_tensor_datatype(global_inp_name, DataType["UINT8"])


model.save(root_dir + "/full_preproc.onnx")


In [132]:
# showInNetron(root_dir + "/full_preproc.onnx")

In [135]:

from qonnx.transformation.insert_topk import InsertTopK

# POST PROC

# insert Top-1 node at the end
model = model.transform(InsertTopK(k=1))
chkpt_name = root_dir + "/pre_post.onnx"
# tidy-up again
model = model.transform(InferShapes())
model = model.transform(FoldConstants())
model = model.transform(GiveUniqueNodeNames())
model = model.transform(GiveReadableTensorNames())
model = model.transform(InferDataTypes())
model = model.transform(RemoveStaticGraphInputs())
model.save(chkpt_name)

# showInNetron(chkpt_name)

In [138]:

from finn.transformation.streamline import Streamline
# we can see the list of apllied transformations here : showSrc(Streamline)
from finn.transformation.streamline.reorder import MoveScalarLinearPastInvariants
import finn.transformation.streamline.absorb as absorb

model = ModelWrapper(root_dir + "/pre_post.onnx")
# STREAMLINE
model = model.transform(Streamline())
model.save(root_dir + "/streamlined.onnx")
# showInNetron(root_dir + "/streamlined.onnx")

In [139]:
from finn.transformation.streamline.round_thresholds import RoundAndClipThresholds
from qonnx.transformation.infer_data_layouts import InferDataLayouts
from qonnx.transformation.general import RemoveUnusedTensors
import finn.transformation.streamline.absorb as absorb

model = ModelWrapper(root_dir + "/streamlined.onnx")

# absorb final add-mul nodes into TopK
model = model.transform(absorb.AbsorbScalarMulAddIntoTopK())
model = model.transform(RoundAndClipThresholds())

# bit of tidy-up
model = model.transform(InferDataLayouts())
model = model.transform(RemoveUnusedTensors())

model.save(root_dir + "/streamlined_merged_and_ready.onnx")
# showInNetron(root_dir + "/streamlined_merged_and_ready.onnx")

In [141]:
import finn.transformation.fpgadataflow.convert_to_hw_layers as to_hw
from qonnx.core.modelwrapper import ModelWrapper

# TO HW LAYERS

model = ModelWrapper(root_dir + "/streamlined_merged_and_ready.onnx")
model = model.transform(to_hw.InferLabelSelectLayer())
model = model.transform(to_hw.InferChannelwiseLinearLayer())
model = model.transform(to_hw.InferQuantizedMatrixVectorActivation())
model = model.transform(to_hw.InferThresholdingLayer())
model.save(root_dir + "/hw.onnx")
showInNetron(root_dir + "/hw.onnx")

Serving '/tmp/finn_dev_parasyte/hw.onnx' at http://0.0.0.0:8081


In [142]:
from qonnx.core.modelwrapper import ModelWrapper

hw_model = ModelWrapper(root_dir + "/hw.onnx")

# listar todos los tipos de nodos que contiene
op_types = set([n.op_type for n in hw_model.graph.node])
print("Tipos de nodos encontrados en hw.onnx:\n", op_types)



Tipos de nodos encontrados en hw.onnx:
 {'Add', 'MVAU', 'Mul', 'Reshape', 'TopK', 'Thresholding', 'Conv', 'MaxPool', 'Transpose'}


In [103]:
from finn.transformation.fpgadataflow.create_dataflow_partition import CreateDataflowPartition

model = ModelWrapper(root_dir + "/hw.onnx")
parent_model = model.transform(CreateDataflowPartition())
parent_model.save(root_dir + "/df_part.onnx")
showInNetron(root_dir + "/df_part.onnx")

# We see that our entire model is now part of a child model in "StreamingDataflowPartition"

AssertionError: cycle-free graph violated: partition depends on itself

In [88]:
from qonnx.custom_op.registry import getCustomOp
sdp_node = parent_model.get_nodes_by_op_type("StreamingDataflowPartition")[0]
sdp_node = getCustomOp(sdp_node)
dataflow_model_filename = sdp_node.get_nodeattr("model")
showInNetron(dataflow_model_filename)

NameError: name 'parent_model' is not defined

In [86]:
# print the names of the supported PYNQ boards
from finn.util.basic import pynq_part_map
print(pynq_part_map.keys())

# change this if you have a different PYNQ board, see list above
pynq_board = "Pynq-Z2"
fpga_part = pynq_part_map[pynq_board]
target_clk_ns = 20

dict_keys(['Ultra96', 'Ultra96-V2', 'Pynq-Z1', 'Pynq-Z2', 'ZCU102', 'ZCU104', 'ZCU111', 'RFSoC2x2', 'RFSoC4x2', 'KV260_SOM'])


In [87]:
from finn.transformation.fpgadataflow.specialize_layers import SpecializeLayers
model = ModelWrapper(dataflow_model_filename)

model = model.transform(SpecializeLayers(fpga_part))

showSrc(SpecializeLayers)

model.save(root_dir + "/to_hw_conv.onnx")
showInNetron(root_dir + "/to_hw_conv.onnx")

NameError: name 'dataflow_model_filename' is not defined